# 少ないデータで転移学習を用いて画像内の表情を感情で分類する


2018.03.29

### ライブラリの読み込み

In [ ]:
%matplotlib inline

import os
import sys
import glob

import keras
import numpy as np
import matplotlib.pyplot as plt

from keras import optimizers
from keras.layers import Dense, Flatten, Input, Activation, add
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.utils import to_categorical

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

# 学習済みネットワーク
from keras.applications.vgg16 import VGG16

### データセットの読み込み
ディレクトリ構成は下記の通り   
各クラス(happy/neutral/angry)30枚の画像を、   
学習(train)20枚/バリデーション(validation)5枚/予測(test)5枚に分割

`dataset`   
`├── train`    
`│   ├── happy`   
`│   ├── neutral `  
`│   └── angry`    
`├── validation`   
`│   ├── happy `  
`│   ├── neutral`   
`│   └── angry`    
`└── test`   
`│   ├── happy `  
`│   ├── neutral`   
`│   └── angry` 

In [ ]:
# 分類
emotions = ['happy', 'neutral', 'angry'] # 分類したい項目名（ディレクトリ名）
emotion_count = len(emotions)

image_width, image_height = 150, 150

# datasetディレクトリ
dataset_path = '(データセットのパスを指定)'

train_data_path = str(dataset_path) + '/train' 
validation_data_path = str(dataset_path) + '/validation' 
test_data_path = str(dataset_path) + '/test' 

# 重みデータを保存するディレクトリ
result_dir = '(重みデータの保存先パスを指定）'

# データの枚数
train_data_count = 20
validation_data_count = 5

# バッチサイズ、エポック数
batch_size = 16
epoch_count = 10

### データの生成（ジェネレータ） 

In [ ]:
# データ生成
datagen = ImageDataGenerator(
    rescale = 1.0/255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.39,
    horizontal_flip=True,
    samplewise_center=False,
    samplewise_std_normalization =False,
    zca_whitening=False)

In [ ]:
generator_train = datagen.flow_from_directory(
    train_data_path,
    target_size=(image_width, image_height),
    color_mode='rgb',
    classes=emotions,
    class_mode='categorical', 
    batch_size=batch_size,
    shuffle=True)

generator_validation = datagen.flow_from_directory(
    validation_data_path, 
    target_size=(image_width, image_height),
    color_mode='rgb',
    classes=emotions,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

### 学習済みモデルの読み込みと全結合層の作成

In [ ]:
# VGG16学習済みモデルの読み込み(RGB)
input_tensor = Input(shape=(image_width, image_height, 3))
learned_model = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# 全結合層の作成
full_model = Sequential()
full_model.add(Flatten(input_shape=learned_model.output_shape[1:]))
full_model.add(Dense(256, activation='relu',
                    kernel_initializer='he_normal'))
full_model.add(Dense(64, activation='relu',
                    kernel_initializer='he_normal'))
full_model.add(Dense(emotion_count, activation='softmax'))

# 学習済みデータと全結合層を結合
model = Model(inputs=learned_model.input, outputs=full_model(learned_model.output))

# 全結合層直前までの層を学習しないようにする
for layer in learned_model.layers[:15]:
    layer.trainable = False

# 学習処理の設定
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
    metrics=['accuracy'])

### 学習

In [ ]:
# 学習
result = model.fit_generator(
    generator_train,
    steps_per_epoch=train_data_count,
    epochs=epoch_count,
    validation_data=generator_validation,
    validation_steps=validation_data_count)

# 重みを保存
model.save_weights(os.path.join(result_dir, 'weight.h5'))

### 学習結果の描画

In [ ]:
fig = plt.figure(figsize=(12,4))

# subplot
# loss
sub_loss = fig.add_subplot(1,2,1)
sub_loss.plot(result.history['loss'])
sub_loss.plot(result.history['val_loss'])
sub_loss.set_title('model loss')
sub_loss.set_ylabel('loss')
sub_loss.set_xlabel('epoch')
sub_loss.legend(['train', 'verify'], loc='upper left')

#Accuracy
sub_acc = fig.add_subplot(1,2,2)
sub_acc.plot(result.history['acc'])
sub_acc.plot(result.history['val_acc'])
sub_acc.set_title('model accuracy')
sub_acc.set_ylabel('accuracy')
sub_acc.set_xlabel('epoch')
sub_acc.legend(['train', 'verify'], loc='upper left')

### テスト画像の予測関数

In [ ]:
def test_predict(filepath_list):

    for i in filepath_list:
        # 画像を読み込んで4次元テンソルへ変換
        img = image.load_img(i, target_size=(image_height, image_width))
        xarray = image.img_to_array(img)
        x = np.expand_dims(xarray, axis=0)
        # テストデータも正規化
        x = x / 255.0

        # 予測 入力は1枚の画像なので[0]のみ
        pred = model.predict(x)[0]
        
        # 描画　topの数だけ予測確率が高い結果を出力
        fig = plt.figure(figsize=(2,2))
        top = 3
        top_indices = pred.argsort()[-top:][::-1]
        result = [(emotions[i], pred[i]) for i in top_indices]
        [print(x) for x in result]
        plt.imshow(img)
        plt.show()
        
        print('------------------------------------')

### 予測

In [ ]:
for x in emotions:
    print('\n\n<<'+ str(x).upper() + '>>\n')
    filepath_list = glob.glob(str(test_data_path)  + '/' + str(x) + '/*')
    test_predict(filepath_list)